In [48]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
%matplotlib inline

In [49]:
df = pd.read_csv('real_estate_data.csv')

C:\Users\admin\AppData\Local\Temp\ipykernel_22148\2314456965.py:1: DtypeWarning: Columns (9,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('real_estate_data.csv')


In [50]:
df

,id,type,sub_type,start_date,end_date,listing_type,tom,building_age,total_floor_count,floor_no,room_count,size,address,furnished,heating_type,price,price_currency
0,1,Konut,Rezidans,12/10/18,1/9/19,2,30,0,20 ve üzeri,2,2+1,90.0,İstanbul/Kartal/Kordonboyu,NaN,Fancoil,3500.0,TRY
1,2,Konut,Daire,2/13/19,NaN,1,14,0,20 ve üzeri,20 ve üzeri,1+0,43.0,İstanbul/Kartal/Kordonboyu,NaN,Fancoil,490000.0,TRY
2,3,Konut,Daire,10/9/18,11/8/18,1,30,0,1,Yüksek Giriş,2+1,NaN,Tekirdağ/Çorlu/Reşadiye,NaN,Fancoil,155000.0,TRY
3,4,Konut,Rezidans,9/10/18,10/10/18,1,30,3,20 ve üzeri,20 ve üzeri,6+1,450.0,İstanbul/Beşiktaş/Levent,NaN,Fancoil,32500000.0,TRY
4,5,Konut,Rezidans,12/10/18,1/9/19,1,30,0,20 ve üzeri,2,2+1,90.0,İstanbul/Kartal/Kordonboyu,NaN,Fancoil,1450000.0,TRY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403482,403483,Konut,Daire,9/18/18,NaN,2,162,NaN,NaN,NaN,+,NaN,İstanbul/Sultanbeyli/Adil,NaN,NaN,1500.0,TRY
403483,403484,Konut,Daire,10/11/18,NaN,1,139,NaN,NaN,NaN,2+1,NaN,Sakarya/Adapazarı/Cumhuriyet,NaN,NaN,120000.0,TRY
403484,403485,Konut,Daire,11/22/18,NaN,1,97,NaN,NaN,NaN,1+1,NaN,Antalya/Alanya/Saray,NaN,NaN,48000.0,EUR
403485,403486,Konut,Daire,2/21/19,NaN,2,6,NaN,NaN,NaN,2+1,2.0,Aydın/Kuşadası/Türkmen,NaN,NaN,900.0,TRY


In [58]:
df.head(10).to_clipboard()

## Описание данных
* id - уникальный идентификатор
* type - тип
* sub_type - подтип
* start_data - дата начала
* end_date - дата конца
* listing_type - тип
* tom - том
* building_age - возраст
* total_floor_count - количество этажей
* floor_no - номер этажа
* room_count - количество комнат
* size - размер
* address - адрес
* furnished - 
* heating_type - тип 
* price - цена
* price_currency - валюта

In [52]:
df = df.drop('start_date', axis=1)
df = df.drop('end_date', axis=1)
df = df.drop('total_floor_count', axis=1)
df = df.drop('floor_no', axis=1)
df = df.drop('address', axis=1)
df = df.drop('furnished', axis=1)
df = df.drop('building_age', axis=1)

In [53]:
df['type'] = df['type'].map({'Konut': 0})

In [54]:
df['sub_type'] = df['sub_type'].map({'Daire': 0, 'Villa': 1, 'Müstakil Ev': 2, 'Rezidans': 3, 'Kooperatif': 4})

In [55]:
X = df.drop('price', axis=1)
y = df['price']